# Gorilla Data Engineer Assessment
## Setup

In [1]:
import platform
print(platform.python_version())

3.11.4


In [17]:
!pip freeze | grep -f requirements.txt

jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.9.0
jupyter-lsp==2.2.1
jupyter_client==8.6.0
jupyter_core==5.5.0
jupyter_server==2.11.1
jupyter_server_terminals==0.4.4
jupyterlab==4.0.9
jupyterlab-widgets==3.0.9
jupyterlab_pygments==0.3.0
jupyterlab_server==2.25.2
matplotlib==3.8.2
matplotlib-inline==0.1.6
numpy==1.26.2
openpyxl==3.1.2
pandas==2.1.3
seaborn==0.13.0


## Load data

In [18]:
import pandas as pd

df_meter_list = pd.read_excel(r'data/gorilla_test_data.xlsx', sheet_name='meter_list')
df_forecast_table = pd.read_excel(r'data/gorilla_test_data.xlsx', sheet_name='forecast_table')
df_rate_table = pd.read_excel(r'data/gorilla_test_data.xlsx', sheet_name='rate_table')

In [20]:
df_meter_list.head()

,meter_id,aq_kwh,exit_zone
0,14676236,28978,EA1
1,34509937,78324,SO1
2,50264822,265667,NT1
3,88357331,484399,SE2


In [21]:
df_forecast_table.head()

,meter_id,date,kwh
0,14676236,2020-06-01,22.070768
1,14676236,2020-06-02,19.170720
2,14676236,2020-06-03,23.555111
3,14676236,2020-06-04,18.220712
4,14676236,2020-06-05,14.196134


In [22]:
df_rate_table.head()

,date,exit_zone,aq_min_kwh,aq_max_kwh,rate_p_per_kwh
0,2020-04-01,EA1,0,73200.0,0.2652
1,2020-04-01,EA1,73200,732000.0,0.1980
2,2020-04-01,EA1,732000,NaN,0.2875
3,2020-04-01,EA2,0,73200.0,0.2970
4,2020-04-01,EA2,73200,732000.0,0.1524
